In [1]:
from randomgen import RandomNetworkGenerator
import json
import requests

In [2]:
schema = json.load(open("../../../metadata/relations.json", "r"))
generator = RandomNetworkGenerator(schema)

custom_counts = {
    "BusinessUnit": 1,
    "ProductFamily": 2,
    "ProductOffering": 3,
    "Facility": 4,
    "Parts": 5,
    "Warehouse": 10,
    "Supplier": 10,
}

In [3]:
API_URL = "http://localhost:8000"

version = "v5"

In [4]:
create_ops = generator.create_network(custom_counts)

print(f"Generated {len(create_ops)} operations")

print(create_ops)

Generated 715 operations
[{'action': 'create', 'type': 'schema', 'payload': {'node_id': '1', 'node_type': 'Parts', 'properties': {'created_at': 0, 'updated_at': 0, 'id': '1', 'name': 'TwPdVQDa', 'description': 'mlaHLSmA', 'type': 'yBgWspRz', 'cost': 995.37, 'importance': 87, 'expected_life': 45, 'units_in_chain': 85, 'expiry': 41}}, 'timestamp': 0}, {'action': 'create', 'type': 'schema', 'payload': {'node_id': '1-1', 'node_type': 'Facility', 'properties': {'created_at': 0, 'updated_at': 0, 'id': '1-1', 'name': 'iuHjRjeW', 'type': 'gQNUskGN', 'location': 'eaJAmdKq', 'max_capacity': 42, 'operating_cost': 222.15}}, 'timestamp': 0}, {'action': 'create', 'type': 'schema', 'payload': {'source_id': '1', 'target_id': '1-1', 'edge_type': 'PartsToFacility', 'properties': {'created_at': 0, 'updated_at': 0, 'quantity': 34, 'distance_from_warehouse': 315.29, 'transport_cost': 632.54, 'lead_time': 74}}, 'timestamp': 0}, {'action': 'create', 'type': 'schema', 'payload': {'node_id': '1-1-1', 'node_typ

In [5]:
updated_create_ops = []

for op in create_ops:
    op["version"] = version
    updated_create_ops.append(op)

requests.post(f"{API_URL}/schema/live/update/bulk", json=updated_create_ops)

<Response [200]>

In [6]:
update_cycles = 2
for _ in range(update_cycles):
    update_ops = generator.generate_updates(
        node_updates=10,
        edge_updates=10
    )
    print(f"Generated {len(update_ops)} update operations")

    updated_update_ops = []

    for op in update_ops:
        op["version"] = version
        updated_update_ops.append(op)

    requests.post(f"{API_URL}/schema/live/update/bulk", json=updated_update_ops)

Generated 14 update operations
Generated 15 update operations


In [7]:
delete_cycles = 0
for _ in range(delete_cycles):
    delete_ops = generator.generate_deletions(
        node_deletions=5,
        edge_deletions=5
    )
    print(f"Generated {len(delete_ops)} delete operations")

    updated_delete_ops = []

    for op in delete_ops:
        op["version"] = version
        updated_delete_ops.append(op)

    requests.post(f"{API_URL}/schema/live/update/bulk", json=updated_delete_ops)